# Decision Tree

Esse modelo serve tanto para tarefas de classificação quanto de regressão, que visa prever valores ou classes de y a partir de decisões simples, inferidas a partir dos dados de treino.  

Tudo começa no nó raiz, onde é escolhido o melhor atributo para dividir os nossos dados. Essa escolha é feita a partir de um critério de impureza, e o resultado disso são conjuntos homogêneos. Ou seja, ele vai avaliar todos os atributos do nosso conjunto de features.  

Depois disso, nós dividimos em outros dois nós, que serão os nós de decisão. A partir deles será tomado outra decisão, e será aplicado de maneira recursiva do critério de verificação de impureza.  

Os nós folhas / terminais, são os nós em que tomamos a decisão. São os nós da nossa variável classificadora.  

Nesse tipo de classificador, não é necessário fazer tratamento nos dados (normalização, limpeza, padronização), pois ela não é tão sensível a essas transformações. Ademais, também não há problemas com binarização de variaveis categoricas, pois a arvore de decisão trabalha tanto com variaveis categóricas quanto numéricas.

Um dos problemas desse modelo é que facilmente ele cai em overfitting, ou seja, ela se adapta perfeitamente aos dados de treino, mas no 'mundo real' acaba não funcionando, pois quando chegam os novos dados ela nao conseguirá se adaptar a esse novo dado.

#### Critério de divisão de nós


- Índice Gini
- Qui-Quadrado
- Ganho de informação (entropia)
- Redução na variância  



Usaremos o índice Gini e a entropia, são os mais utilizados e mais conhecidos.

- *O índice Gini:*  
  
  Este índice informa o grau de heterogeneidade dos dados. O objetivo dele é medir a frequência de um elemento aleatório de um nó ser rotulado de maneira incorreta.

  G = somatório da probabilidade de uma classe ocorrer * (1 - probabilidade da classe ocorrer). Aplicando o índice gini nas nossas features, a feature com menor indice gini será a característica conhecida como nó raiz, a que melhor dividirá nossos dados.

- *A entropia:*  
  
  A entropia mede a ordem dos dados por meio da variável classificadora. Ela foca no ganho de informação. Se um dado está muito desordenado, então ele ta impuro, logo menor a possibilidade de ele ser o nó raiz. Quanto maior a entropia, maior a desordem

  entropy = - somatório da probabilidade de uma classe ocorrer * log da probabilidade da classe ocorrer

#### Importando dados

In [2]:
import pandas as pd

uri = "https://raw.githubusercontent.com/alura-cursos/ML_Classificacao_por_tras_dos_panos/main/Dados/Customer-Churn.csv"

dataset = pd.read_csv(uri)
dataset.shape

(7043, 18)

#### Binarizando informações categóricas

In [3]:
swap = {
  "Sim": 1,
  "Nao": 0
}

columns = ["Conjuge", "Dependentes", "TelefoneFixo", "PagamentoOnline", "Churn"]
modified_dataset = dataset[columns].replace(swap)
modified_dataset.head()


,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn
0,1,0,0,1,0
1,0,0,1,0,0
2,0,0,1,1,1
3,0,0,0,0,0
4,0,0,1,1,1


In [4]:
# aqui estamos removendo as colunas que não queremos modificar

dummie_data = pd.get_dummies(dataset.drop(columns=columns))
# get_dummies vai identificar as categorias presentes nessas colunas e a partir de um loop vai pegar
# todas essas categorias presentes nas colunas e transformar em novas colunas, contendo 1(uns) e 0(zeros).

# Juntando os dois conjuntos com os dados transformados
transformed_dataset = pd.concat([dummie_data, modified_dataset], axis=1)
transformed_dataset.head()

,Maior65Anos,MesesDeContrato,ContaMensal,VariasLinhasTelefonicas_Nao,VariasLinhasTelefonicas_SemServicoTelefonico,VariasLinhasTelefonicas_Sim,ServicoDeInternet_DSL,ServicoDeInternet_FibraOptica,ServicoDeInternet_Nao,SegurancaOnline_Nao,...,TipoDeContrato_UmAno,FormaDePagamento_CartaoDeCredito,FormaDePagamento_ChequeDigital,FormaDePagamento_ChequePapel,FormaDePagamento_DebitoEmConta,Conjuge,Dependentes,TelefoneFixo,PagamentoOnline,Churn
0,0,1,29.85,0,1,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0
1,0,34,56.95,1,0,0,1,0,0,0,...,1,0,0,1,0,0,0,1,0,0
2,0,2,53.85,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,1,1
3,0,45,42.30,0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
4,0,2,70.70,1,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,1,1


In [5]:
# Separando as features das labels

x = transformed_dataset.drop("Churn", axis = 1)
y = transformed_dataset["Churn"]

In [6]:
from imblearn.over_sampling import SMOTE

smt = SMOTE(random_state=123)

# aplicando o oversampling, gerando novos dados 'fakes'
# aliás, pelo que reparei, tem que binarizar tudo pra usar esse resampling
x, y = smt.fit_resample(x, y)
transformed_dataset = pd.concat([x, y], axis = 1)

#### Definindo treino e teste

In [7]:
from sklearn.model_selection import train_test_split
import numpy as np

SEED = 42
np.random.seed(SEED)

# lembra que aqui tem de se passar o x normalizado, e não o x original
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, stratify=y)

#### Importando e treinando o modelo

In [8]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(criterion="entropy")
model.fit(train_x, train_y)

DecisionTreeClassifier(criterion='entropy')

#### Verificar a importância de cada atributo

In [9]:
model.feature_importances_

array([1.66779181e-02, 1.60919972e-01, 2.17887580e-01, 1.43465523e-02,
       2.87982802e-03, 1.11871318e-02, 2.01495366e-03, 3.39299175e-02,
       0.00000000e+00, 1.39153249e-02, 2.08900831e-03, 1.50561375e-02,
       7.33181310e-03, 2.49417233e-03, 8.79027866e-03, 8.25317687e-03,
       1.04823792e-04, 9.75394659e-03, 1.09917547e-02, 1.59029174e-04,
       1.78936260e-02, 9.00258417e-03, 0.00000000e+00, 2.62290983e-03,
       9.94397315e-03, 2.77720300e-04, 1.37557519e-02, 3.66981572e-02,
       1.61618337e-01, 7.61137306e-02, 1.71808829e-02, 1.68717032e-02,
       1.63103573e-02, 1.89846171e-02, 2.21568221e-02, 1.66872493e-02,
       5.08630891e-03, 2.00119497e-02])

#### Validando o modelo

In [10]:
from sklearn.metrics import accuracy_score

predictions = model.predict(test_x)
accuracy = accuracy_score(test_y, predictions) * 100

# accuracy = model.score(test_x, test_y)

print(f"The accuracy of BernoulliNB model in {len(test_x)} test elements was {accuracy:.2f}%")

The accuracy of BernoulliNB model in 3105 test elements was 80.81%


In [11]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(test_y, predictions))

[[1232  321]
 [ 275 1277]]


##### Precision

In [12]:
from sklearn.metrics import precision_score

precision = precision_score(test_y, predictions) * 100
print(f"The precision of Decision Tree in {len(test_x)} elements was {precision:.2f}%")

The precision of Decision Tree in 3105 elements was 79.91%


##### Recall

In [14]:
from sklearn.metrics import recall_score

recall = recall_score(test_y, predictions) * 100
print(f"The recall of Decision Tree in {len(test_x)} elements was {recall:.2f}%")

The recall of Decision Tree in 3105 elements was 82.28%
